In [42]:
%autosave 2
from multiprocessing import Pool, cpu_count
import numpy as np
from functools import partial

n_threads = cpu_count()

Autosaving every 2 seconds


In [24]:
n=1000001
def f(n=1000001):
    a = np.random.normal(scale=1, size=n)
    return np.mean(a)

In [25]:
%%timeit
for _ in range(n_threads):
    f()

422 ms ± 410 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
with Pool(processes=n_threads) as p:
    a = p.map(f, [1000001]*n_threads)

128 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
def sphere_volume(n, rndstate=None):
    if rndstate is None:
        a = np.random.uniform(-1, 1, size=(n, 3))
    else:
        rndstate.uniform(-1, 1, size=(n, 3))
    a_bool = np.sum(np.square(a), axis=1) < 1
    k = np.sum(a_bool)
    return k

n = int(1e7)
%timeit print((sphere_volume(n) / n * 8) / (4/3*np.pi))

0.999738395877
1.00037132326
1.00038602917
0.999736677004
0.999971398714
0.999604896711
1.00021108606
1.00010317901
950 ms ± 44.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
with Pool(processes=n_threads) as p:
    v = p.map(sphere_volume, [n//n_threads]*n_threads)
print(sum(v) / n * 8 / (4/3*np.pi))

0.999906081525
0.999906081525
0.999906081525
0.999906081525
0.999906081525
0.999906081525
0.999906081525
0.999906081525
290 ms ± 49.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
args = [(n//n_threads, np.random.RandomState())
        for _ in range(n_threads)]
with Pool(processes=n_threads) as p:
    v = p.starmap(
        sphere_volume,
        args
    )
print(sum(v) / n * 8 / (4/3*np.pi))

UnboundLocalError: local variable 'a' referenced before assignment